In [ ]:
'''
This notebook submits the jobs to aws batch using imported generated Latin hypercube sampling inputs generated from another notebook.

It does not do a good job of monitoring the batch jobs, so your best bet is to log into the aws console and monitor the jobs there. 
I'm working on it, but it's not quite there yet.

There's a variable set to True or False towards the end that you'll want to start with as False and change to True when jobs are done.
After that, it will download all of the outfile.lis.csv files and then create a single .tgz file out of all of them in the outputs directory.
'''

import boto3
import matrices.LHS_1000  # make sure the size is correct!


matrix = matrices.LHS_1000.matrix  # here too!
print(len(matrix))

In [ ]:
size = len(matrix)
size

In [ ]:
# experiment name/number
exp = 'soybeans'

In [ ]:
client = boto3.client('batch')
client

In [ ]:
jobs = []
jobs

In [ ]:
jobs_uri = "s3://daycent-jobs/jobs/{}_{}/jobs".format(exp, size)
jobs_uri
# s3://daycent-jobs/jobs/corn_1000/diffs/diff_0.tgz

In [ ]:
# s3://daycent-jobs/jobs/corn_1000/diffs/diff_0.tgz

In [ ]:
%%time

# submit the jobs to aws batch 
# really ought to use a threadpool for submitting these (TODO)

for i in range(size):  # size
    job_name = 'test_{}_{}_{}'.format(exp, size, i)
    response = client.submit_job(
        jobDefinition='test-job-def',
        jobName=job_name,
        jobQueue='queue',
        containerOverrides={
          'command': [ 
              "-s","soybeans",
              "-n","soybeans",
              "-e","soybeans_prev",
              "-f","s3://daycent-jobs/jobs/soybeans/soybeans_prev.bin",  # bin to extend from
              "-l","yes",  # run list100 after
              "-i","s3://daycent-jobs/jobs/soybeans/soybeans.tgz",  # full input directory
              "-j", jobs_uri,  # job s3 bucket
              # "-o","yes",  # if you want to download the complete output directory rather than just the output file
              "-d","s3://daycent-jobs/jobs/{}_{}/diffs/diff_{}.tgz".format(exp, size, i),  # make sure this makes sense
              "-r","{}".format(i),
              #"-c", "yes" # capture stderr and stdout to daycent.log.txt and list100.log.txt
          ],  
        },
    )
    jobs.append(response)
    # print(response)
print("Submitted {} jobs.".format(len(jobs)))

In [ ]:
print("num jobs is {}".format(len(jobs)))

In [ ]:
job_listing = client.list_jobs(
    jobQueue = 'queue',
    jobStatus='RUNNABLE',
)
job_listing
#print("Job list gave {} jobs".format(len(job_listing)))

In [ ]:
ids = []
for job in jobs:
    ids.append(job['jobId'])
print(len(ids))

In [ ]:
jobstuff = client.describe_jobs(
    jobs=ids[-100:]
)
for job in jobstuff['jobs']:
    print("name: {}, status: {}".format(job['jobName'], job['status']))

In [ ]:
# randomly samples a subset of jobs and prints their status to the output every few seconds for a few iterations
# clears the previous output before printing the new one
# you can just run this cell over and over again to get an idea of the jobs, not perfect, but it's somewhere to start

import time
import random
from IPython.display import clear_output

iterations = 40  # number of times to pull jobs and get their status
num_jobs = 75 # number of jobs to randomly select from to view
sleep_time = 5  # number of seconds to sleep between job polling

for i in range(iterations):
    done = True
    jobstuff = client.describe_jobs(
        jobs=list(random.sample(ids, num_jobs))
    )
    # jobstuff
    clear_output(wait=True)
    output = ""
    for stuff in jobstuff['jobs']:
        output = output + "i={} : {} : {}\n".format(i, stuff['jobName'], stuff['status'])
    print(output, end='\r')
    time.sleep(sleep_time)
print('done')

In [ ]:
import os

print(os.getcwd())
if not os.path.isdir('outputs/{}_{}'.format(exp, size)):
    os.makedirs('outputs/{}_{}'.format(exp, size))

In [ ]:
# flip this to True when the jobs are finished running, or put in False to pause here while they're still running
jobs_done = True
jobs_done

In [ ]:
%%time

# copy all the outfile.lis.csv files back to the outputs directory named to correspond to their job number
# really ought to use a threadpool for submitting these (TODO)

import subprocess

# just holding things up here to wait for the jobs to finish since I can't check them all because aws doesn't like checking on more than
# 100 jobs at a time. Change to jobs_done True to continue
if jobs_done: 
    
    for i in range(len(matrix)):
        # grab the csv files and stick them somewhere
        # cmd should look like:
        # aws s3 cp s3://daycent-jobs/jobs/exp1000/jobs/{}/outfile.lis.csv ./outputs/exp1000/outfile_{}.lis.csv
        subprocess.call(['/home/jovyan/aws-cli/v2/current/bin/aws', 's3', 'cp', 
                         '{}/{}/outfile.lis.csv'.format(jobs_uri, i), 
                         './outputs/{}_{}/outfile_{}.list.csv'.format(exp, size, i)])
print("sure done, why not.")

In [ ]:
%%time

# create a single tgz file containing all of the outfile_{job}.lis.csv files

# just holding things up here to wait for the jobs to finish since I can't check them all because aws doesn't like checking on more than
# 100 jobs at a time. Change jobs_done to True to continue
if jobs_done:
    import os

    os.chdir('outputs')
    print(os.getcwd())

    # tgz them all up
    subprocess.call(['tar', 'czvf', '{}_{}.tgz'.format(exp, size), '{}_{}'.format(exp, size)])
    print('done tgzing them')